# Predicción con Series de Tiempo

En este laboratorio se implementan 5 métodos diferentes para predecir con series de tiempo, utilizando 4 data sets distintos.

In [1]:
# se cargan las librerias necesarias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# se cargan los data sets
df_1 = pd.read_csv("daily-total-female-births.csv")
df_2 = pd.read_csv("monthly-car-sales.csv")
df_3 = pd.read_csv("monthly-mean-temp.csv")
df_4 = pd.read_csv("shampoo.csv")

In [20]:
df_1.head()

,Date,Births
0,1959-01-01,35
1,1959-01-02,32
2,1959-01-03,30
3,1959-01-04,31
4,1959-01-05,44


In [21]:
df_2.head()

,Month,Sales
0,1960-01,6550
1,1960-02,8728
2,1960-03,12026
3,1960-04,14395
4,1960-05,14587


In [22]:
df_3.head()

,Month,Temperature
0,1920-01,40.6
1,1920-02,40.8
2,1920-03,44.4
3,1920-04,46.7
4,1920-05,54.1


In [23]:
df_4.head()

,Month,Sales
0,1-01,266.0
1,1-02,145.9
2,1-03,183.1
3,1-04,119.3
4,1-05,180.3


#### 1. Promedio

In [5]:
def prediccion_promedio(historia, config):
    n, tipo_prom = config
    if tipo_prom is "prom":
        return mean(historia[-n:])
    return median(historia[-n:])

<>:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-5-34b0f1facfc2>:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if tipo_prom is "prom":


#### 2. SARIMAX

In [7]:
def prediccion_sarima(historia, orden, orden_estacional, tendencia):
    modelo = SARIMAX(historia, order = orden, seasona_order = orden_estacional,
                     trend = tendencia, enforce_stationarity = False, enforce_invertibility = False)
    modelo_ajustado = modelo.fit(disp = False)
    yhat = modelo_ajustado.predict(len(historia), len(historia))
    return yhat[0]

#### 3. Winter-Holt

In [8]:
def AlisamientoExponencial(historia, t, d, s, p, b, r):
    historia = array(historia)
    modelo = ExponentialSmoothing(historia, trend = t, damped = d,
                                  seasonal = s, seasonal_periods = p)
    modelo_ajustado = modelo.fit(optimized = True, use_boxcox = b, remove_bias = r)
    yhat = modelo_ajustado.predict(len(historia), len(historia))
    return yhat[0]

#### 4. Red Neuronal

In [16]:
#Transformación de series
def series_a_supervisado(datos, n_entrada, n_salida = 1):
    df = DataFrame(datos)
    columnas = [ ]
    for i in range(n_entrada, 0, -1):
        columnas.append(df.shift(-1))
    for i in range(0, n_salida):
        columnas.appemd(df.shift(-1))
    agregado = concat(columnas, axis = 1)
    agregado.dropna(inplace = true)
    return agregado.values

#Predicción con Tensorflow y Keras
def prediccion_red_neuronal(historia, n_entrada, n_nodos, n_epocas, tamanio_tandas):
    datos = series_a_supervisado(historia, n_entrada)
    entreno_X, entreno_y = datos[:, :-1], datos[:, -1]
    modelo = Sequential()
    modelo.add(Dense(n_nodes, activation='relu', input_dim=n_entrada))
    modelo.add(layers.Dense(1))
    modelo.compile(loss = "mse", optimizer = "adam")
    modelo_ajustado = modelo.fit(entreno_X, entreno_y, epochs = n_epocas,
    batch_size = tamanio_tandas, verbose = 0)
    x_entrada = array(historia[-n_entrada:]).reshape(1, n_entrada)
    yhat = modelo.predict(x_entrada, verbose = 0)
    return yhat[0]

#### 5. FB Prophet

In [24]:
#Requiere de una pequeña transformación de los datos a usar
series = pd.read_csv("monthly-car-sales.csv", header = 0, index_col = None)
series["ds"] = pd.to_datetime(series["Month"])
series["y"] = series[["Sales"]].astype(float)
series = series[["ds", "y"]]
series.head( )

#Ejecución
modelo = Prophet(mcmc_samples = 500, seasonality_mode = "multiplicative")
modelo_ajustado = modelo.fit(series)
futuro = modelo_ajustado.make_future_dataframe(periods = 48, freq = "M")
prediccion = modelo_ajustado.predict(futuro)
prediccion[["ds", "yhat", "yhat_lower", "yhat_upper"]].tail( )

NameError: name 'Prophet' is not defined